# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException



ws = Workspace.from_config()

In [ ]:
# Create compute cluster
cluster_name = "compute-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Existing Compute using {cluster_name}")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=6, min_nodes=1)
    compute_target = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)

## Dataset

### Overview
This dataset comes from data extracted from the US Census of 1994. The purpose is to be able to predict whether a person makes a salry of more than $50K per year. The specific features were extracted from a the broader 1994 census data. The dataet was retrieved from kaggle: https://www.kaggle.com/datasets/ayessa/salary-prediction-classification.

Dataset Features:
* **age**: The age of the person (int)
* **workclass**: The class of job for the person (string category)
* **fnlwgt**: Weight assigned by US Census Bureau for number of people represented by that entry (int)
* **education**: The type of education someone has received (string category)
* **education-num**: A nunerical representation of the education column (int)
* **marital-status**: The marital status of a person (string category)
* **occupation**: The occucpation of a person (string category)
* **relationship**: The family relationship of a person (string category)
* **race**: The ethnicity of a person in the report (string category)
* **sex**: The gender of the person in the report (string category)
* **capital-gain**: The capital gains of a person in the report (int)
* **capital-loss**: The capital losses of a person in the report (int)
* **hour-per-week**: Hours worked per work (int)
* **native-country**: The native country for the person in the report (int)
* **salary**: The salary <=50K or >50K we are trying to predict (category)


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
from azureml.core.dataset import Dataset
from train import prep_data
# choose a name for experiment
experiment_name = 'capstone-automl'

experiment=Experiment(ws, experiment_name)
dataset = Dataset.get_by_name(ws, name='salary')
df = dataset.to_pandas_dataframe()
X, y = prep_data(df)

X['Salary'] = y

ds = ws.get_default_datastore()

train_dataset = Dataset.Tabular.register_pandas_dataframe(X, ds, 'automl-dataset')
X.head()

## AutoML Configuration

* **task**: Set to 'classification' since our goal is to classify a person's salary
* **experiment_timeout_minutes**: Set to 30 minutes to get results in a timely manner and to not waste resources
* **primary_metric**: Set to 'accuracy' because its a reasonable metric to determine the outcome in a classification
* **label_column**: Set to the result column from the dataset, the salary being more or less than 50k, which is what is being predicted
* **compute_target**: The compute cluster to run the experiment on
* **n_cross_validations**: Set to 5 to remove bias when doing validation
* **enable_early_stopping**: Terminate the experiment early if the score isn't improving
* **max_concurrent_iterations**: Allow for multiple runs to be done in parallel

In [ ]:
from azureml.train.automl import AutoMLConfig
# TODO: Put your automl settings here
automl_settings = {
  "max_concurrent_iterations": 5
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
  task="classification",
  experiment_timeout_minutes=30,
  training_data=train_dataset,
  primary_metric='accuracy',
  label_column_name='Salary',
  compute_target=compute_target,
  n_cross_validations=5,
  enable_early_stopping=True,
  **automl_settings
)

In [ ]:
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails
# TODO: Submit your experiment
remote_run: AutoMLRun = experiment.submit(automl_config)
RunDetails(remote_run).show()

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save best automl model
best_model, model = remote_run.get_output()
print(best_model)
print(model.steps)
registered_model = remote_run.register_model()
registered_model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core import Environment
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice

env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies_automl.yml')
inference_config = InferenceConfig(
    environment=env,
    source_directory=".",
    entry_script="./score.py",
)
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

service = Model.deploy(
    ws,
    "automlservice2",
    [registered_model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
